# Reading files

In [10]:
from google.colab import files
uploaded=files.upload()

Saving orders_cleaned.csv to orders_cleaned.csv


In [5]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("orders_analysis").getOrCreate()
spark

In [6]:
cf=spark.read.csv('customers_cleaned.csv',header=True,inferSchema=True)
cf.show()


+---+-----------+-------------+--------------------+--------+--------------------+
|_c0|customer_id|         name|               email|   phone|             address|
+---+-----------+-------------+--------------------+--------+--------------------+
|  0|          1|Alice Johnson|alice.johnson@exa...|555-0101|123 Maple St, Spr...|
|  1|          2|    Bob Smith|bob.smith@example...|555-0202|456 Oak St, Orlan...|
|  2|          3|    Carol Lee|carol.lee@example...|555-0303|789 Pine St, Seat...|
|  3|          4|  David Patel|david.patel@examp...|555-0404|101 Birch St, Bos...|
|  4|          5|     Eva Wang|eva.wang@example.com|555-0505|202 Cedar St, Pho...|
+---+-----------+-------------+--------------------+--------+--------------------+



In [11]:
df=spark.read.csv('deliveries_cleaned.csv',header=True,inferSchema=True)
df.show()

+---+---------+--------+---------+-------------------+--------------------+
|_c0|status_id|order_id|   status|        status_time|             remarks|
+---+---------+--------+---------+-------------------+--------------------+
|  0|        1|    1001|  pending|2025-07-02 08:00:00|      left warehouse|
|  1|        2|    1001|  shipped|2025-07-03 09:30:00|  departed warehouse|
|  2|        3|    1001|delivered|2025-07-05 14:15:00|  signed by customer|
|  3|        4|    1002|  pending|2025-07-04 10:00:00|      order received|
|  4|        5|    1002|  shipped|2025-07-05 11:00:00|picked up by carrier|
|  5|        6|    1002|  delayed|2025-07-08 16:00:00|       weather delay|
|  6|        7|    1003|  pending|2025-07-06 07:45:00|      order received|
|  7|        8|    1003|  shipped|2025-07-07 12:20:00|          in transit|
|  8|        9|    1003|delivered|2025-07-09 09:10:00|  left at front door|
|  9|       10|    1004|  pending|2025-07-08 08:30:00|      order received|
| 10|       

In [12]:
of=spark.read.csv('orders_cleaned.csv',header=True,inferSchema=True)
of.show()

+---+--------+-----------+----------+------------+-------------+
|_c0|order_id|customer_id|order_date|total_amount|delivery_date|
+---+--------+-----------+----------+------------+-------------+
|  0|    1001|          1|2025-07-01|       120.5|   2025-07-03|
|  1|    1002|          2|2025-07-03|        75.0|   2025-07-05|
|  2|    1003|          3|2025-07-05|       200.0|   2025-07-07|
|  3|    1004|          1|2025-07-07|        50.0|   2025-07-09|
|  4|    1005|          4|2025-07-10|      315.75|   2025-07-12|
|  5|    1006|          2|2025-07-11|       99.99|   2025-07-13|
|  6|    1007|          5|2025-07-12|       150.0|   2025-07-14|
|  7|    1008|          3|2025-07-14|       65.25|   2025-07-16|
|  8|    1009|          4|2025-07-15|       230.0|   2025-07-17|
|  9|    1010|          5|2025-07-16|        45.0|   2025-07-18|
+---+--------+-----------+----------+------------+-------------+



# Joining orders with customer tables

In [18]:
oc=of.join(cf.select("customer_id","name","email","phone","address"),on='customer_id',how='left')

oc=oc.join(df.select("order_id","status_id","status","status_time","remarks"),on="order_id",how="right")
oc=oc.select("_c0","order_id","customer_id","status_id","order_date","delivery_date","status","name","total_amount","email","phone","status_time","address","remarks")
oc.show()

+---+--------+-----------+---------+----------+-------------+---------+-------------+------------+--------------------+--------+-------------------+--------------------+--------------------+
|_c0|order_id|customer_id|status_id|order_date|delivery_date|   status|         name|total_amount|               email|   phone|        status_time|             address|             remarks|
+---+--------+-----------+---------+----------+-------------+---------+-------------+------------+--------------------+--------+-------------------+--------------------+--------------------+
|  0|    1001|          1|        1|2025-07-01|   2025-07-03|  pending|Alice Johnson|       120.5|alice.johnson@exa...|555-0101|2025-07-02 08:00:00|123 Maple St, Spr...|      left warehouse|
|  0|    1001|          1|        2|2025-07-01|   2025-07-03|  shipped|Alice Johnson|       120.5|alice.johnson@exa...|555-0101|2025-07-03 09:30:00|123 Maple St, Spr...|  departed warehouse|
|  0|    1001|          1|        3|2025-07-0

In [52]:
from pyspark.sql.functions import col,sum,split,count
oc=oc.withColumn("region",split(col("address"),",")[len('address')-4])
oc.show()
oc_filtered=oc.filter(col("status")=="delayed")
oc_filtered.show()
grouped_region=oc_filtered.groupBy("region").count().alias("grouped_region")
grouped_region.show()

+---+--------+-----------+---------+----------+-------------+---------+-------------+------------+--------------------+--------+-------------------+--------------------+--------------------+-----------+
|_c0|order_id|customer_id|status_id|order_date|delivery_date|   status|         name|total_amount|               email|   phone|        status_time|             address|             remarks|     region|
+---+--------+-----------+---------+----------+-------------+---------+-------------+------------+--------------------+--------+-------------------+--------------------+--------------------+-----------+
|  0|    1001|          1|        1|2025-07-01|   2025-07-03|  pending|Alice Johnson|       120.5|alice.johnson@exa...|555-0101|2025-07-02 08:00:00|123 Maple St, Spr...|      left warehouse|    Midwest|
|  0|    1001|          1|        2|2025-07-01|   2025-07-03|  shipped|Alice Johnson|       120.5|alice.johnson@exa...|555-0101|2025-07-03 09:30:00|123 Maple St, Spr...|  departed warehous

# saving the result

In [53]:
grouped_region.write.csv("grouped_region.csv",header=True)

In [54]:
oc.write.csv("joined_tables.csv",header=True)
